In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
len(mnist.test.images)

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
D = 784
x = tf.placeholder(tf.float32, [None, D])

In [ ]:
W = tf.Variable(tf.zeros([D, 10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])

# Solution from Tutorial

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [ ]:
# tf.argmax(y,1) - what we predicted
# tf.argmax(y_,1) - correct label
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# Custom solution

See: https://github.com/pilipolio/schibsted-study/blob/master/notebooks/201612_mnist.ipynb

### Task
* Re-user softmax classification from https://github.com/pilipolio/schibsted-study/blob/master/notebooks/201611_multiclass_classification.ipynb
* Define train_X and train_Y from mnist_data.train.images/labels
* Calculate the accuracy (ratio of correctly classified labels)
* Think about a way to visualise the fitted weights
* Improve the accuracy of the naive model by adding non-linear layers (see https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

WIDTH, HEIGHT = (28, 28)

# unique_labels = np.unique(mnist.train.labels)
unique_labels = 10
# print('Train has {} dimensions with {} unique classe: {}'.format(mnist.train.images.shape, len(unique_labels), unique_labels))

In [ ]:
n_samples, D = mnist.train.images.shape
C = unique_labels #.shape[0]

n_samples, D, C

In [ ]:
def show_image(pixels):
    plt.imshow(pixels.reshape((WIDTH, HEIGHT)), cmap='Greys', interpolation='None');
    plt.xticks([]); plt.yticks([]);

show_image(mnist.train.images[0])

In [ ]:
# Parameters
learning_rate = .5
n_epochs = 1000
display_step = 100

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
init = tf.initialize_all_variables()

In [ ]:
# storing weights for each epochs for later visualisation
fitted_ws = np.zeros((n_epochs, D * unique_labels))

for epoch in range(n_epochs):

    batch_xs, batch_ys = mnist.train.next_batch(100)
    # batch_xs = mnist.train.images
    # batch_ys = mnist.train.labels
    _, fitted_w, loss_value, predicted_probs = sess.run(
        fetches=[optimizer, W, cross_entropy, y],
        feed_dict={x: batch_xs, y_: batch_ys})
    fitted_ws[epoch, :] = fitted_w.ravel()
    predicted_probs = sess.run(y, feed_dict={x: batch_xs})
    predicted_classes = predicted_probs.argmax(axis=1)
    n_correct_samples = np.sum(predicted_classes == batch_ys.argmax(axis=1))
    percent_correct = round(n_correct_samples / len(predicted_classes) * 100)
    
    if (epoch) % display_step == 0:
        print("Epoch: {:4d}, cost={:.4f}, %correct {}".format(epoch, loss_value, percent_correct))

In [ ]:
w_images = fitted_ws[-1].reshape((D, 10))
show_image(w_images[:,6])

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# Conv-net solution using TF Slim

In [ ]:
import tensorflow.contrib.slim as slim

In [ ]:
def lenet(images):
   net = slim.conv2d(images, 20, [5,5], scope='conv1')
   net = slim.max_pool2d(net, [2,2], scope='pool1')
   net = slim.conv2d(net, 50, [5,5], scope='conv2')
   net = slim.max_pool2d(net, [2,2], scope='pool2')
   net = slim.flatten(net, scope='flatten3')
   net = slim.fully_connected(net, 500, scope='fully_connected4')
   net = slim.fully_connected(net, 10, activation_fn=None, scope='fully_connected5')
   return net

In [ ]:
mnist.train.images.reshape((-1, HEIGHT, WIDTH, 1)).shape

In [ ]:
images = mnist.train.images.reshape((-1, HEIGHT, WIDTH, 1))
net = lenet(images)
g = tf.Graph()

In [ ]:
labels = tf.one_hot(tf.argmax(mnist.train.labels, 1), 10, dtype=tf.int32)
loss = slim.losses.softmax_cross_entropy(net, labels)

In [ ]:
log_dir = './log/train' # Where checkpoints are stored.
learning_rate=.001

total_loss = slim.losses.get_total_loss()
# tf.summary.scalar('losses/total_loss', total_loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

# create_train_op ensures that each time we ask for the loss, the update_ops
# are run and the gradients being computed are applied too.
train_op = slim.learning.create_train_op(total_loss, optimizer)

In [ ]:
slim.learning.train(
    train_op,
    log_dir,
    number_of_steps=1,
    save_summaries_secs=300,
    save_interval_secs=600)